In [ ]:
import os
import shutil
from pathlib import Path
from tqdm import tqdm
import re

In [ ]:
names = [
    "ClustalO",
    "Mafft_FFT-NS-2",
    "Mafft_L-INS-i",
    "Mafft_G-INS-i",
    "Muscle3",
    "Muscle5",
]
names_new = names_new = [
    "ClustalO",
    "MAFFT_FFT-NS-2",
    "MAFFT_L-INS-i",
    "MAFFT_G-INS-i",
    "Muscle3",
    "Muscle5",
]
name_map = dict(zip(names, names_new))

regex_default = re.compile(r"^msa\.[0-9]+\..*\.fasta$$")
regex_muscle5 = re.compile(r"^[a-z]{3,4}\.[0-9]+\.msa\.fasta$")

In [ ]:
base_dir = Path("/hits/fast/cme/bodynems/data/output/treebase_v2/")
print(len(os.listdir(base_dir)))
counts = dict(zip(names, [0] * len(names)))
missing = []
for dataset in tqdm(os.listdir(base_dir)):
    data_dir = base_dir / dataset
    if os.path.isdir(data_dir):
        count = 0
        for name in names:
            ens_dir = data_dir / name
            regex = regex_muscle5 if name == "Muscle5" else regex_default
            count += len(
                list(
                    filter(
                        lambda msa_file: re.match(regex, msa_file), os.listdir(ens_dir)
                    )
                )
            )
        if count != 48:
            missing.append(data_dir)
            print(data_dir)
            # for msa_name in filter(lambda msa_file: re.match(regex, msa_file), os.listdir(ens_dir)):
            #     counts[name] += 1

# Three datasets are incomplete, filter out

3122


 49%|████▉     | 1524/3122 [00:03<00:03, 464.62it/s]

/hits/fast/cme/bodynems/data/output/treebase_v2/10271_0.phy


 92%|█████████▏| 2880/3122 [00:06<00:00, 481.52it/s]

/hits/fast/cme/bodynems/data/output/treebase_v2/10271_2.phy
/hits/fast/cme/bodynems/data/output/treebase_v2/15191_23.phy


100%|██████████| 3122/3122 [00:06<00:00, 472.74it/s]


In [ ]:
# Copy msa files to ensemble directories in dest_dir
dest_dir = Path("/hits/fast/cme/bodynems/data/paper/treebase_v1")
for dataset in tqdm(os.listdir(base_dir)):
    data_dir = base_dir / dataset
    if os.path.isdir(data_dir) & (data_dir not in missing):
        os.makedirs(dest_dir / dataset / "ensemble", exist_ok=True)

        if len(os.listdir(dest_dir / dataset / "ensemble")) == 48:
            continue

        for name in names:
            ens_dir = data_dir / name
            regex = regex_muscle5 if name == "Muscle5" else regex_default
            msa_files = sorted(os.listdir(ens_dir))

            for i, msa_name in enumerate(
                filter(lambda msa_file: re.match(regex, msa_file), msa_files)
            ):
                dest_path = (
                    dest_dir
                    / dataset
                    / "ensemble"
                    / ".".join([name_map[name], str(i), "fasta"])
                )
                shutil.copy(ens_dir / msa_name, dest_path)

  0%|          | 0/3122 [00:00<?, ?it/s]

100%|██████████| 3122/3122 [00:02<00:00, 1083.98it/s]


In [ ]:
# Add "done" files
dest_dir = Path("/hits/fast/cme/bodynems/data/paper/treebase_v1")
for dataset in tqdm(os.listdir(base_dir)):
    if dataset not in missing:
        data_dir = dest_dir / dataset
        for name in names_new:
            os.makedirs(dest_dir / dataset / name, exist_ok=True)
            open(dest_dir / dataset / name / "done", "a").close()

  0%|          | 0/3122 [00:00<?, ?it/s]

100%|██████████| 3122/3122 [00:33<00:00, 92.01it/s]
